<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#소-가격-크롤링" data-toc-modified-id="소-가격-크롤링-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>소 가격 크롤링</a></span></li><li><span><a href="#돼지-가격-크롤링" data-toc-modified-id="돼지-가격-크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>돼지 가격 크롤링</a></span></li><li><span><a href="#local-mysql에-적재하는-코드-짜놓기" data-toc-modified-id="local-mysql에-적재하는-코드-짜놓기-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>local mysql에 적재하는 코드 짜놓기</a></span><ul class="toc-item"><li><span><a href="#소-DB에-적재하는-코드" data-toc-modified-id="소-DB에-적재하는-코드-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>소 DB에 적재하는 코드</a></span></li><li><span><a href="#돼지-DB에-적재" data-toc-modified-id="돼지-DB에-적재-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>돼지 DB에 적재</a></span></li></ul></li></ul></div>

- 출처 : 축산물품질평가원 공공 데이터 포탈 API이용

- <소>
    - 소 권역별 경락가격 현황
- <돼지>
    - 돼지 박피,탕박 권역별 경락가격 현황


In [1]:
# 한 번에 가져오는 건 일일 트래픽 1000을 초과하기 때문에 2년치 씩만 가져와서 DB에 적재하자..
# 오늘 밤 12시 지나서 하기

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
beef_url = 'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/cattleApperence?baseYmd=2017-09-10&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
pork_url = 'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/pigApperence?baseYmd=20170908&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'

## 소 가격 크롤링

In [2]:
beef_dict = {}

dates = []
beefs = []
beef_ks = []
regions = []
prices = []

datetimes = pd.date_range(start='2021-03-24', end='2021-03-24', freq='D').strftime('%Y-%m-%d').tolist()

for datetime in datetimes:
    beef_url = f'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/cattleApperence?baseYmd={datetime}&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
    res = requests.get(beef_url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.select('item')
    print(datetime)
    for content in contents:
        date = content.select_one('baseymd').get_text()
        beef = content.select_one('judgekindnm').get_text()
        beef_k = content.select_one('judgebreednm').get_text()
        region = content.select_one('localnm').get_text()
        price = content.select_one('auctamt').get_text()
        
        dates.append(date)
        beefs.append(beef)
        beef_ks.append(beef_k)
        regions.append(region)
        prices.append(price)
        
beef_dict['날짜'] = dates
beef_dict['동물'] = beefs
beef_dict['종류'] = beef_ks
beef_dict['지역'] = regions
beef_dict['가격'] = prices

2021-03-24


In [3]:
pd.options.display.max_rows = 100
df = pd.DataFrame(beef_dict)
df.head(20)

,날짜,동물,종류,지역,가격
0,20210324,소,한우,수도권,19297
1,20210324,소,한우,중부권,20213
2,20210324,소,한우,전국,19739
3,20210324,소,육우,수도권,11844
4,20210324,소,육우,중부권,10727
5,20210324,소,육우,전국,11767


---

## 돼지 가격 크롤링

In [7]:
pork_dict = {}

dates = []
porks = []
pork_ks = []
regions = []
prices = []

datetimes = pd.date_range(start='2021-01-01', end='2021-03-24', freq='D').strftime('%Y%m%d').tolist()
for datetime in datetimes:
    pork_url = f'http://data.ekape.or.kr/openapi-data/service/user/grade/auct/pigApperence?baseYmd={datetime}&ServiceKey=0OhBU7ZCGIobDVKDeBJDpmDRqK3IRNF6jlf%2FJB2diFAf%2FfR2czYO9A4UTGcsOwppV6W2HVUeho%2FFPwXoL6DwqA%3D%3D'
    res = requests.get(pork_url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.select('item')
    print(datetime)
    for content in contents:
        date = content.select_one('baseymd').get_text()
        pork = content.select_one('judgekindnm').get_text()
        pork_k = content.select_one('skinnm').get_text()
        region = content.select_one('localnm').get_text()
        price = content.select_one('auctamt').get_text()
        
        dates.append(date)
        porks.append(pork)
        pork_ks.append(pork_k)
        regions.append(region)
        prices.append(price)
    
pork_dict['날짜'] = dates
pork_dict['동물'] = porks
pork_dict['종류'] = pork_ks
pork_dict['지역'] = regions
pork_dict['가격'] = prices

20210101
20210102
20210103
20210104
20210105
20210106
20210107
20210108
20210109
20210110
20210111
20210112
20210113
20210114
20210115
20210116
20210117
20210118
20210119
20210120
20210121
20210122
20210123
20210124
20210125
20210126
20210127
20210128
20210129
20210130
20210131
20210201
20210202
20210203
20210204
20210205
20210206
20210207
20210208
20210209
20210210
20210211
20210212
20210213
20210214
20210215
20210216
20210217
20210218
20210219
20210220
20210221
20210222
20210223
20210224
20210225
20210226
20210227
20210228
20210301
20210302
20210303
20210304
20210305
20210306
20210307
20210308
20210309
20210310
20210311
20210312
20210313
20210314
20210315
20210316
20210317
20210318
20210319
20210320
20210321
20210322
20210323
20210324


In [8]:
df = pd.DataFrame(pork_dict)
df

,날짜,동물,종류,지역,가격
0,20201231,돼지,탕박,수도권,3757
1,20201231,돼지,탕박,영남권,3600
2,20201231,돼지,탕박,전국,3682
3,20201231,돼지,탕박,수도권,3757
4,20201231,돼지,탕박,영남권,3600
...,...,...,...,...,...
296,20210323,돼지,탕박,영남권,4151
297,20210323,돼지,탕박,전국,4607
298,20210324,돼지,탕박,수도권,4437
299,20210324,돼지,탕박,영남권,4185


## local mysql에 적재하는 코드 짜놓기

- 2015년 01월 07일 까지 DB 적재 완료 상태!

### 소 DB에 적재하는 코드

In [6]:
import pymysql
db = pymysql.connect(host='localhost',
                    port=3306,
                    user='younghun',
                    password='watson1259',
                    db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

rows = map(list, zip(*beef_dict.values()))

# 한 row씩 DB에 insert하기
for row in rows:
    sql = """INSERT INTO beef_prices VALUES('"""+row[0]+"""','"""+row[1]+"""','"""+row[2]+"""',
    '"""+str(row[3])+"""','"""+str(row[4])+"""')"""
    cursor.execute(sql)
    db.commit()
    
db.close()

### 돼지 DB에 적재

In [9]:
import pymysql
db = pymysql.connect(host='localhost',
                    port=3306,
                    user='younghun',
                    password='watson1259',
                    db='beef_pork_db',
                    charset='utf8')
cursor = db.cursor()

rows = map(list, zip(*pork_dict.values()))

# 한 row씩 DB에 insert하기
for row in rows:
    sql = """INSERT INTO pork_prices VALUES('"""+row[0]+"""','"""+row[1]+"""','"""+row[2]+"""',
    '"""+str(row[3])+"""','"""+str(row[4])+"""')"""
    cursor.execute(sql)
    db.commit()
    
db.close()

---